In [29]:
import requests

In [30]:
FINALES = "http://127.0.0.1:13371"
USERNAME = "dummy"
PASSWORD = "password"

# Authenticate user

In [31]:
response = requests.post(
    f"{FINALES}/user_management/authenticate",
    data={
        "grant_type": "password",
        "username": USERNAME,
        "password": PASSWORD,
        "scope": "",
        "client_id": "string",
        "client_secret": "string",
    },
    headers={
        "accept": "application/json",
        "Content-Type": "application/x-www-form-urlencoded",
    },
)

if not response.ok:
    raise requests.exceptions.HTTPError(f"Failed to authenticate: {response.text}")

data: dict[str, str] = response.json()

headers = {
    "accept": "application/json",
    "Content-Type": "application/x-www-form-urlencoded",
    "Authorization": "{} {}".format(
        data["token_type"].capitalize(),
        data["access_token"],
    ),
}

# Check FINALES capabilities

In [33]:
response = requests.get(
    f"{FINALES}/capabilities",
    headers=headers,
)

response.ok

True

# Test measurement tenant

In [36]:
import time


response = requests.post(
    f"{FINALES}/requests",
    json={
        "quantity": "RGB-measurement",
        "methods": [
            "RGB-simulation",
        ],
        "parameters": {
            "RGB-simulation": {
                "R": 30,
                "G": 100,
                "B": 80,
                "warmup_time": 1,
            }
        },
        "tenant_uuid": "",
    },
    headers=headers,
)

print(response.json())

# while True:
#     response = requests.get(f"{FINALES}/requests/{request_id}", headers=headers)

#     if response.json()["status"] == "resolved":
#         response = requests.get(f"{FINALES}/results/{request_id}")
#         break

#     time.sleep(15)

# measured_rgb = response.json()["data"]["rgb-measurement"]

# id, R, G, B = measured_rgb

# print(f"RGB measurement for request {id}: [{R}, {G}, {B}]")

{'detail': [{'type': 'model_attributes_type', 'loc': ['body'], 'msg': 'Input should be a valid dictionary or object to extract fields from', 'input': '{"quantity": "RGB-measurement", "methods": ["RGB-simulation"], "parameters": {"RGB-simulation": {"R": 30, "G": 100, "B": 80, "warmup_time": 1}}, "tenant_uuid": ""}'}]}


In [ ]:
TARGET = (35, 137, 94)  # Dark green

In [ ]:
import time


request_id = requests.post(
    f"{FINALES}/POST/request",
    json={
        "quantity": "RGB-prediction",
        "methods": [
            "RGB-optimization",
        ],
        "parameters": {
            "RGB-optimization": {
                "R": TARGET[0],
                "G": TARGET[1],
                "B": TARGET[2],
                "n_calls": 50,
            }
        },
    },
    headers=headers,
)

while True:
    response = requests.get(f"{FINALES}/GET/requests/{request_id}")

    if response.json()["status"] == "resolved":
        response = requests.get(f"{FINALES}/GET/results/{request_id}")
        break

    time.sleep(15)

predicted_rgb = response.json()["data"]["rgb-prediction"]

id, minimum, cost = predicted_rgb

print(
    (
        "Predicted RGB:",
        f"  target: {TARGET}",
        f"  predicted: {minimum}",
        f"  cost: {cost}",
    )
)